In [1]:
pip install face_recognition opencv-python numpy

     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.5/100.1 MB 2.4 MB/s eta 0:00:42
      --------------------------------------- 1.3/100.1 MB 2.7 MB/s eta 0:00:37
      --------------------------------------- 1.8/100.1 MB 2.7 MB/s eta 0:00:37
     - -------------------------------------- 3.1/100.1 MB 3.5 MB/s eta 0:00:28
     - -------------------------------------- 3.9/100.1 MB 3.6 MB/s eta 0:00:28
     - -------------------------------------- 4.7/100.1 MB 3.6 MB/s eta 0:00:27
     -- ------------------------------------- 5.5/100.1 MB 3.6 MB/s eta 0:00:26
     -- ------------------------------------- 6.3/100.1 MB 3.7 MB/s eta 0:00:26
     -- ------------------------------------- 7.1/100.1 MB 3.7 MB/s eta 0:00:26
     --- ------------------------------------ 7.9/100.1 MB 3.6 MB/s eta 0:00:26
     --- ------------------------------------ 8.9/100.

In [3]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

class FaceRecognitionSystem:
    def __init__(self, known_faces_dir: str, tolerance: float = 0.6):
        """Initialize the face recognition system.
        
        Args:
            known_faces_dir: Directory containing known face images
            tolerance: Threshold for face recognition confidence (lower is stricter)
        """
        self.known_face_encodings = []
        self.known_face_names = []
        self.tolerance = tolerance
        
        if not os.path.exists(known_faces_dir):
            os.makedirs(known_faces_dir)
            print(f"Created directory for known faces: {known_faces_dir}")
        
        self.load_known_faces(known_faces_dir)
        
    def load_known_faces(self, directory: str) -> None:
        """Load and encode known faces from the specified directory."""
        print(f"\nLoading known faces from {directory}")
        start_time = datetime.now()
        
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(directory, filename)
                name = os.path.splitext(filename)[0].replace('_', ' ').title()
                
                try:
                    image = face_recognition.load_image_file(image_path)
                    encodings = face_recognition.face_encodings(image)
                    
                    if len(encodings) > 0:
                        self.known_face_encodings.append(encodings[0])
                        self.known_face_names.append(name)
                        print(f"Loaded face encoding for: {name}")
                    else:
                        print(f"No face found in image: {filename}")
                        
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
        
        duration = (datetime.now() - start_time).total_seconds()
        print(f"\nLoaded {len(self.known_face_names)} faces in {duration:.2f} seconds")
        
    def recognize_face(self, face_encoding: np.ndarray) -> Tuple[str, float]:
        """Compare a face encoding with known faces and return the best match."""
        if not self.known_face_encodings:
            return "Unknown", 0.0
            
        # Calculate face distances
        face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        confidence = 1 - face_distances[best_match_index]
        
        # Check if the best match is within tolerance
        if confidence >= (1 - self.tolerance):
            return self.known_face_names[best_match_index], confidence
        
        return "Unknown", confidence
    
    def process_image(self, image: np.ndarray) -> Tuple[np.ndarray, List[str]]:
        """Process an image and recognize faces."""
        # Convert BGR to RGB for face_recognition library
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        recognized_names = []
        
        try:
            # Find all faces in the image
            face_locations = face_recognition.face_locations(rgb_image)
            face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
            
            # Process each detected face
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Recognize the face
                name, confidence = self.recognize_face(face_encoding)
                if name != "Unknown":
                    recognized_names.append(name)
                
                # Draw the face box
                cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
                
                # Draw the name label
                label_background = (left, bottom - 35, right, bottom)
                cv2.rectangle(image, (label_background[0], label_background[1]),
                            (label_background[2], label_background[3]),
                            (0, 255, 0), cv2.FILLED)
                
                # Add name and confidence
                label_text = f"{name} ({confidence:.1%})"
                cv2.putText(image, label_text, (left + 6, bottom - 6),
                           cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
                
        except Exception as e:
            print(f"Error during face recognition: {str(e)}")
            
        return image, recognized_names
    
    def process_video_stream(self, video_source: int = 0) -> None:
        """Process video stream and recognize faces in real-time."""
        print(f"\nInitializing video capture from source: {video_source}")
        
        try:
            cap = cv2.VideoCapture(video_source)
            if not cap.isOpened():
                raise ValueError(f"Error opening video source {video_source}")
            
            # Get video properties
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            
            print(f"Video dimensions: {frame_width}x{frame_height}")
            print(f"Frame rate: {fps} FPS")
            print("\nPress 'q' to stop recognition...")
            
            frame_count = 0
            start_time = datetime.now()
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process the frame
                processed_frame, recognized_names = self.process_image(frame)
                
                # Display statistics
                cv2.putText(processed_frame, f"Recognized: {len(recognized_names)}",
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                cv2.imshow('Face Recognition', processed_frame)
                frame_count += 1
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        except Exception as e:
            print(f"Error during video processing: {str(e)}")
            
        finally:
            if 'cap' in locals():
                duration = (datetime.now() - start_time).total_seconds()
                actual_fps = frame_count / duration
                
                print(f"\nVideo processing completed:")
                print(f"Processed {frame_count} frames in {duration:.2f} seconds")
                print(f"Average processing speed: {actual_fps:.2f} FPS")
                
                cap.release()
                cv2.destroyAllWindows()

def main():
    known_faces_dir = "known_faces"
    
    try:
        recognizer = FaceRecognitionSystem(known_faces_dir)
        recognizer.process_video_stream()
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()


Loading known faces from known_faces
Loaded face encoding for: Eduardo-Barrios-Xmcllggmmyu-Unsplash
Loaded face encoding for: Juan-Pablo-Valdez-0Acbatow5To-Unsplash
Loaded face encoding for: Michael-Dam-Mez3Pofgs K-Unsplash

Loaded 3 faces in 48.15 seconds

Initializing video capture from source: 0
Video dimensions: 640x480
Frame rate: 0 FPS

Press 'q' to stop recognition...

Video processing completed:
Processed 67 frames in 34.02 seconds
Average processing speed: 1.97 FPS
